## Selecting Relevant Corpora

In [11]:
import pymongo
from pymongo import DeleteMany
from pymongo.errors import BulkWriteError

In [12]:
db = pymongo.MongoClient().doc_store
total_corpora_articles = db.test.count()

/home/de-admin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


In [13]:
import pickle

In [14]:
cat_doc_map = pickle.load(open("/home/de-admin/Documents/open data/wikipedia category document map/cat doc map", "rb" ))

In [16]:
minimum_articles = 500
minimum_average_words_per_article = 1000
minimum_words = minimum_articles * minimum_average_words_per_article
maximum_corpora_proportion = 0.001
maximum_corpora_articles = total_corpora_articles*maximum_corpora_proportion

final_article_corpora = []
articles_removed = 0
words_removed = 0

for (outer_k, outer_v) in cat_doc_map.items():
    number = outer_v['number']
    words = outer_v['words']
    
    if number > minimum_articles:
        article_dist = outer_v['article_dist']
        number_down = 0
        words_down = 0
        articles_to_use = []
        
        for (inner_k, inner_v) in article_dist.items():
            if int(inner_k) < minimum_average_words_per_article:
                number_down = number_down + inner_v[0]
                words_down = words_down + (inner_k*inner_v[0])
            else:
                for article in inner_v[1]:
                    articles_to_use.append(article)
        
        final_number = number - number_down
        final_words = words - words_down
        
        if final_number <= maximum_corpora_articles:
            if final_number >= minimum_articles:
                if final_words >= minimum_words:
                    final_article_corpora.append([outer_k, final_number, final_words, articles_to_use])
                else:
                    pass
            else:
                pass
        else:
                pass
    else:
        pass

## Building Text Classifiers 

In [17]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


### Split selected corpora into test-training lists 

In [18]:
# create a dataframe using texts and lables
trainDF = pandas.DataFrame(final_article_corpora, columns=["label","number articles","number words","titles"])

#sampling the titles per category
train_titles = []
train_titles_n = []
test_titles = []
test_titles_n = []

for row in trainDF['titles']:
    output = model_selection.train_test_split(row, test_size=0.1)
    train_titles.append(output[0])
    train_titles_n.append(len(output[0]))
    test_titles.append(output[1])
    test_titles_n.append(len(output[1]))

trainDF["train titles"] = train_titles
trainDF["train titles count"] = train_titles_n
trainDF["test titles"] = test_titles
trainDF["test titles count"] = test_titles_n

print("Total Categories: ",len(trainDF),"\nLargest category articles: ",trainDF.iloc[trainDF["number articles"].values.argmax()]["number articles"],"\nLargest category words: ",trainDF.iloc[trainDF["number words"].values.argmax()]["number words"],"\n\nTotal Articles: ", trainDF['number articles'].sum(),"\nTotal train articles: ",trainDF['train titles count'].sum(),"\nTotal test articles: ",trainDF['test titles count'].sum())

Total Categories:  540 
Largest category articles:  5052 
Largest category words:  23429096 

Total Articles:  556967 
Total train articles:  501033 
Total test articles:  55934


## Define Custom Vocabulary

In [19]:
minimum_occurances = 50000
maximum_length = 10

final_vocab = []

for word in db.vocab.find({ "count": { "$gt": minimum_occurances } }):
    if len(word["_id"]) > maximum_length:
        pass
    else:
        final_vocab.append(word["_id"])
        
len(final_vocab)

6977

### Count Vectoriser

In [22]:
#Make a generator to call documents one at a time from mongo

def make_corpus():
    i=0
    for row in trainDF["train titles"]:
        for title in row:
            yield db.test.find_one({'_id': title })["text"]
        i = i + 1
        if i % 10 == 0:
            print(i)
    
corpus = make_corpus()

In [23]:
#Define count vectoriser and fit selected training corpus

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', vocabulary=final_vocab)
count_vect.fit(corpus)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390


MemoryError: 

In [ ]:
# transform the training and validation data using count vectorizer object

xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)